#### 一. RDD重用  
1. 对于多次使用的中间RDD, 应该对其缓存, 以避免重复计算  
  1. 直接调用cache()或者presist()方法对指定的RDD进行缓存（持久化）操作
  1. 如下, RDD2应该被缓存
  <img src="../img/rdd1.png" height="70%" width="70%">
  2. 缓存级别  
    ```bash
    MEMORY_ONLY       #数据全部缓存在内存中 
    MEMORY_ONLY_2     #数据以双副本的方式缓存在内存中
    MEMORY_ONLY_SER   #数据全部以序列化的方式缓存到内存中
    MEMORY_AND_DISK            #数据一部分缓存在内存中，一部分持久化到磁盘上
    MEMORY_AND_DISK_SER        #数据以序列化的方式一部分缓存在内存中，一部分持久化到磁盘上
    MEMORY_AND_DISK_2          #数据以双副本的方式一部分缓存到内存中，一部分持久化到磁盘上
    DISK_ONLY                  #数据全部持久化到磁盘上
    ```

#### 二. 算子调优 
1. `使用map side预聚合的shuffle`  
　　reduceByKey或者aggregateByKey算子都允许用户指定分区内的聚合函数和分区间的聚合函数. 分区内的聚合函数就是map端的预聚合, 和hadoop中的Combiner类似. 而groupByKey则不会提供分区内的聚合, 应避免使用  

2. `使用mapPartitions代替map算子`  
　　mapPartitions类的算子，一次函数调用会处理一个partition所有的数据，而不是一次函数调用处理一条，性能相对来说会高一些。但是有的时候，使用mapPartitions会出现OOM（内存溢出）的问题。因为单次函数调用就要处理掉一个partition所有的数据，如果内存不够，垃圾回收时是无法回收掉太多对象的，很可能出现OOM异常。所以使用这类操作时要慎重！
```scala
val a = sc.parallelize(1 to 9, 3)
  def doubleFunc(iter: Iterator[Int]) : Iterator[(Int,Int)] = {
    var res = List[(Int,Int)]()
    while (iter.hasNext)
    {
      val cur = iter.next;
      res .::= (cur,cur*2)
    }
    res.iterator
  }
val result = a.mapPartitions(doubleFunc)
println(result.collect().mkString)
```
3. `如果有数据库连接操作之类, 使用foreachPartitions代替foreach`  
　　原理类似于“使用mapPartitions替代map”，也是一次函数调用处理一个partition的所有数据，而不是一次函数调用处理一条数据。在实践中发现，foreachPartitions类的算子，对性能的提升还是很有帮助的。比如在foreach函数中，将RDD中所有数据写MySQL，那么如果是普通的foreach算子，就会一条数据一条数据地写，每次函数调用可能就会创建一个数据库连接，此时就势必会频繁地创建和销毁数据库连接，性能是非常低下；但是如果用foreachPartitions算子一次性处理一个partition的数据，那么对于每个partition，只要创建一个数据库连接即可，然后执行批量插入操作，此时性能是比较高的。实践中发现，对于1万条左右的数据量写MySQL，性能可以提升30%以上。
  
4. `使用filter之后进行coalesce操作`  
　　通常对一个RDD执行filter算子过滤掉RDD中较多数据后（比如30%以上的数据），建议使用coalesce算子，手动减少RDD的partition数量，将RDD中的数据压缩到更少的partition中去。因为filter之后，RDD的每个partition中都会有很多数据被过滤掉，此时如果照常进行后续的计算，其实每个task处理的partition中的数据量并不是很多，有一点资源浪费，而且此时处理的task越多，可能速度反而越慢。因此用coalesce减少partition数量，将RDD中的数据压缩到更少的partition之后，只要使用更少的task即可处理完所有的partition。在某些场景下，对于性能的提升会有一定的帮助。  
  
5. `repartitionAndSortWithPartitions代替先repartition再sort`  
　　repartitionAndSortWithinPartitions是Spark官网推荐的一个算子，官方建议，如果需要在repartition重分区之后，还要进行排序，建议直接使用repartitionAndSortWithinPartitions算子。因为该算子可以一边进行重分区的shuffle操作，一边进行排序。shuffle与sort两个操作同时进行，比先shuffle再sort来说，性能可能是要高的。

#### 三. 广播变量大变量  
1. Executor内部使用`CachedThreadPool`执行代码, 如果每个线程都使用了Driver机器上的共有变量, 则每个线程都会从从Driver拷贝数据, 从而产生大量网络IO  
2. 使用广播变量后, 每个Executor都会把共有变量从Driver上拷贝到自己的`BlockManager`上, 此后Executor开启的每个线程只要从BlockManager中获取即可, 减少网络IO  
3. 一般当RDD大于100M就要考虑广播变量

#### 四. 使用Kryo优化序列化性能
1. 以下情况, Sprak会对数据序列化后传输  
  1. 分区数据shuffle后在网络间传输时  
  2. 持久化RDD时
2. 使用Kyro序列化提升速度  
Spark默认使用的是Java的序列化机制，也就是ObjectOutputStream/ObjectInputStream API来进行序列化和反序列化。但是Spark同时支持使用Kryo序列化库，Kryo序列化类库的性能比Java序列化类库的性能要高很多。官方介绍，Kryo序列化机制比Java序列化机制，性能高10倍左右。Spark之所以默认没有使用Kryo作为序列化类库，是因为Kryo要求最好要注册所有需要进行序列化的自定义类型，因此对于开发者来说，这种方式比较麻烦。  
以下是使用Kryo的代码示例，我们只要设置序列化类，再注册要序列化的自定义类型即可（比如算子函数中使用到的外部变量类型、作为RDD泛型类型的自定义类型等）：
```scala
// 创建SparkConf对象。
val conf = new SparkConf().setMaster(...).setAppName(...)
// 设置序列化器为KryoSerializer。
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
// 注册要序列化的自定义类型。
conf.registerKryoClasses(Array(classOf[MyClass1], classOf[MyClass2]))
```